# FX Risk Management for Treasury

## Use Case Overview

**Business Problem:** Oil & Gas refiners have significant FX exposure:
- **Crude purchases**: USD-denominated payables
- **Product sales**: Local currency (TRY) receivables
- **Result**: Structural USD short position

**AI Applications:**
1. Exposure forecasting and aggregation
2. FX rate prediction (short-term)
3. Optimal hedge ratio recommendation
4. Natural hedge identification (oil-USD correlation)

---

## Learning Objectives

1. Calculate and visualize FX exposures
2. Analyze FX rate patterns and volatility
3. Build hedge effectiveness simulation
4. Create FX risk dashboard

In [ ]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys
sys.path.append('..')
from src.treasury_sim.generators import generate_fx_rates, generate_fx_exposures, set_seed

print("✅ Setup complete!")

## 1. Generate FX Data

In [ ]:
set_seed(42)

# Generate FX rates (1 year daily)
fx_rates = generate_fx_rates(days=365, currency_pairs=['USD/TRY', 'EUR/TRY', 'EUR/USD'], seed=42)

# Generate exposures
exposures = generate_fx_exposures(n_exposures=200, seed=42)

print(f"📊 FX Rates: {len(fx_rates):,} records")
print(f"📊 Exposures: {len(exposures):,} positions")

In [ ]:
# Preview data
print("FX RATES (Sample):")
display(fx_rates.head())

print("\nEXPOSURES (Sample):")
display(exposures.head())

## 2. FX Rate Analysis

### Thinking Trace 🧠

> **Key metrics for FX risk:**
> - **Volatility**: Standard deviation of returns
> - **VaR**: Value at Risk (95th percentile loss)
> - **Correlation**: Between currency pairs

In [ ]:
# Pivot for analysis
fx_pivot = fx_rates.pivot(index='timestamp', columns='currency_pair', values='mid')

# Calculate returns
fx_returns = fx_pivot.pct_change().dropna()

# Statistics
print("📊 FX VOLATILITY ANALYSIS (Annualized)")
print("=" * 50)
vol_stats = pd.DataFrame({
    'Daily Vol (%)': fx_returns.std() * 100,
    'Annual Vol (%)': fx_returns.std() * np.sqrt(252) * 100,
    '95% VaR (Daily %)': fx_returns.quantile(0.05) * 100,
    'Max Drawdown (%)': ((fx_pivot / fx_pivot.cummax()) - 1).min() * 100
})
print(vol_stats.round(2))

In [ ]:
# Visualize FX rates
fig = make_subplots(rows=2, cols=2,
    subplot_titles=('USD/TRY Rate', 'EUR/TRY Rate', 'Daily Returns Distribution', 'Correlation Matrix'))

# Rate charts
for i, pair in enumerate(['USD/TRY', 'EUR/TRY']):
    row, col = (1, i+1)
    fig.add_trace(
        go.Scatter(x=fx_pivot.index, y=fx_pivot[pair], name=pair, mode='lines'),
        row=row, col=col
    )

# Returns distribution
for pair in ['USD/TRY', 'EUR/TRY']:
    fig.add_trace(
        go.Histogram(x=fx_returns[pair]*100, name=pair, opacity=0.7, nbinsx=50),
        row=2, col=1
    )

# Correlation heatmap
corr = fx_returns.corr()
fig.add_trace(
    go.Heatmap(z=corr.values, x=corr.columns, y=corr.index, colorscale='RdBu', zmid=0),
    row=2, col=2
)

fig.update_layout(height=600, title_text='FX Rate Analysis', showlegend=False)
fig.show()

## 3. Exposure Analysis

In [ ]:
# Exposure summary by currency
exposure_summary = exposures.groupby('currency').agg({
    'amount_local': ['sum', 'count', 'mean'],
    'is_hedged': 'mean'
}).round(2)

exposure_summary.columns = ['Net Exposure', 'Count', 'Avg Size', 'Hedge Ratio']

print("📊 EXPOSURE SUMMARY BY CURRENCY")
print("=" * 60)
print(exposure_summary)

In [ ]:
# Exposure by maturity bucket
maturity_summary = exposures.groupby(['currency', 'maturity_bucket'])['amount_local'].sum().unstack(fill_value=0)

fig = px.bar(exposures.groupby(['maturity_bucket', 'currency'])['amount_local'].sum().reset_index(),
             x='maturity_bucket', y='amount_local', color='currency', barmode='group',
             title='FX Exposures by Maturity Bucket')
fig.show()

## 4. Hedge Effectiveness Analysis

### Thinking Trace 🧠

> **What is hedge effectiveness?**
> - Measures how well a hedge offsets exposure
> - Perfect hedge = -1 correlation with exposure
> - IFRS 9 requires 80-125% effectiveness for hedge accounting

In [ ]:
# Simulate hedge P&L
def simulate_hedge_pnl(exposure_usd, hedge_ratio, fx_start, fx_end):
    """
    Simulate P&L for hedged vs unhedged position.
    
    Exposure: USD payables (we're short USD)
    Hedge: Forward contract to buy USD
    """
    # Unhedged P&L (exposure to FX movement)
    fx_change = (fx_end - fx_start) / fx_start
    unhedged_pnl = exposure_usd * fx_change  # Loss if USD strengthens
    
    # Hedged amount locked in at fx_start
    hedged_amount = exposure_usd * hedge_ratio
    hedge_pnl = -hedged_amount * fx_change  # Gain if USD strengthens
    
    # Net P&L
    net_pnl = unhedged_pnl + hedge_pnl
    
    return {
        'Unhedged P&L': unhedged_pnl,
        'Hedge P&L': hedge_pnl,
        'Net P&L': net_pnl,
        'Effectiveness': abs(hedge_pnl / unhedged_pnl) * 100 if unhedged_pnl != 0 else 0
    }

# Example simulation
exposure = 10_000_000  # $10M USD payable
fx_start = 34.0  # USD/TRY at start
fx_end = 36.0    # USD/TRY at end (TRY weakened)

scenarios = []
for hedge_ratio in [0, 0.25, 0.50, 0.75, 1.0]:
    result = simulate_hedge_pnl(exposure, hedge_ratio, fx_start, fx_end)
    result['Hedge Ratio'] = f"{hedge_ratio:.0%}"
    scenarios.append(result)

scenario_df = pd.DataFrame(scenarios)[['Hedge Ratio', 'Unhedged P&L', 'Hedge P&L', 'Net P&L', 'Effectiveness']]

print("📊 HEDGE EFFECTIVENESS SIMULATION")
print(f"Scenario: USD/TRY moves from {fx_start} to {fx_end} (+{(fx_end/fx_start-1)*100:.1f}%)")
print(f"Exposure: ${exposure:,.0f} USD payables")
print("=" * 70)
print(scenario_df.to_string(index=False))

In [ ]:
# Monte Carlo simulation for optimal hedge ratio
n_simulations = 1000
exposure = 10_000_000
fx_start = 34.0

# Simulate FX movements (normal distribution based on historical vol)
annual_vol = fx_returns['USD/TRY'].std() * np.sqrt(252)
daily_vol = annual_vol / np.sqrt(252)
horizon_days = 90
horizon_vol = daily_vol * np.sqrt(horizon_days)

fx_simulations = fx_start * (1 + np.random.normal(0, horizon_vol, n_simulations))

# Calculate P&L for different hedge ratios
hedge_ratios = np.linspace(0, 1, 21)
results = []

for hr in hedge_ratios:
    pnls = []
    for fx_end in fx_simulations:
        result = simulate_hedge_pnl(exposure, hr, fx_start, fx_end)
        pnls.append(result['Net P&L'])
    
    pnl_array = np.array(pnls)
    results.append({
        'Hedge Ratio': hr,
        'Mean P&L': pnl_array.mean(),
        'Std P&L': pnl_array.std(),
        'VaR 95%': np.percentile(pnl_array, 5),
        'Max Loss': pnl_array.min()
    })

mc_results = pd.DataFrame(results)

# Visualize
fig = make_subplots(rows=1, cols=2,
    subplot_titles=('P&L Volatility by Hedge Ratio', 'VaR (95%) by Hedge Ratio'))

fig.add_trace(
    go.Scatter(x=mc_results['Hedge Ratio']*100, y=mc_results['Std P&L']/1e6,
               mode='lines+markers', name='P&L Std Dev'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=mc_results['Hedge Ratio']*100, y=mc_results['VaR 95%']/1e6,
               mode='lines+markers', name='VaR 95%', line=dict(color='red')),
    row=1, col=2
)

fig.update_xaxes(title_text='Hedge Ratio (%)')
fig.update_yaxes(title_text='$ Million', row=1, col=1)
fig.update_yaxes(title_text='$ Million', row=1, col=2)
fig.update_layout(height=400, title_text='Optimal Hedge Ratio Analysis (Monte Carlo)')
fig.show()

# Find optimal
optimal_idx = mc_results['VaR 95%'].idxmax()  # Maximize (least negative) VaR
print(f"\n💡 OPTIMAL HEDGE RATIO: {mc_results.loc[optimal_idx, 'Hedge Ratio']:.0%}")
print(f"   Expected VaR (95%): ${mc_results.loc[optimal_idx, 'VaR 95%']/1e6:.2f}M")

## 5. FX Risk Dashboard

In [ ]:
# Create executive dashboard
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        'Net Exposure by Currency', 'USD/TRY Rate (YTD)', 'Exposure Maturity Profile',
        'Hedge Coverage', 'VaR Trend', 'FX P&L Impact'
    ),
    specs=[[{"type": "pie"}, {"type": "scatter"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "scatter"}, {"type": "waterfall"}]]
)

# 1. Net exposure pie
exp_by_ccy = exposures.groupby('currency')['amount_local'].sum().abs()
fig.add_trace(go.Pie(labels=exp_by_ccy.index, values=exp_by_ccy.values), row=1, col=1)

# 2. FX rate
fig.add_trace(go.Scatter(x=fx_pivot.index, y=fx_pivot['USD/TRY'], mode='lines'), row=1, col=2)

# 3. Maturity profile
mat_profile = exposures.groupby('maturity_bucket')['amount_local'].sum()
fig.add_trace(go.Bar(x=mat_profile.index, y=mat_profile.values/1e6), row=1, col=3)

# 4. Hedge coverage
hedge_cov = exposures.groupby('currency')['is_hedged'].mean() * 100
fig.add_trace(go.Bar(x=hedge_cov.index, y=hedge_cov.values), row=2, col=1)
fig.add_hline(y=75, line_dash="dash", line_color="green", row=2, col=1, annotation_text="Target")

# 5. Rolling VaR
rolling_var = fx_returns['USD/TRY'].rolling(30).apply(lambda x: np.percentile(x, 5)) * exposure / 1e6
fig.add_trace(go.Scatter(x=rolling_var.index, y=rolling_var.values, mode='lines'), row=2, col=2)

# 6. P&L waterfall (mock)
fig.add_trace(go.Waterfall(
    x=['Opening', 'FX Movement', 'Hedge Gain', 'Closing'],
    y=[0, -2.5, 1.8, -0.7],
    measure=['absolute', 'relative', 'relative', 'total']
), row=2, col=3)

fig.update_layout(height=600, title_text='FX Risk Management Dashboard', showlegend=False)
fig.show()

## 6. Key Takeaways

### AI Applications in FX Risk

| Application | Model | Benefit |
|-------------|-------|--------|
| **Exposure Forecasting** | LSTM/Prophet | Better cash flow prediction |
| **Rate Prediction** | Ensemble (limited value) | Short-term tactical |
| **Hedge Optimization** | Monte Carlo + RL | Optimal hedge ratios |
| **Anomaly Detection** | Isolation Forest | Identify unusual exposures |

### Business Recommendations

1. **Maintain 75-100% hedge** for core exposures
2. **Use options** for uncertain exposures (forecasts)
3. **Monitor oil-USD correlation** for natural hedge
4. **Automate exposure aggregation** from multiple sources

---

*Author: Ozgur Guler (ozgur.guler1@gmail.com)*